In [1]:
import requests
import pandas as pd
import numpy as np
import os
import sqlite3
from tqdm.auto import tqdm
from bs4 import BeautifulSoup


# Rubric
## Creating an ETL guided by https://medium.com/analytics-vidhya/building-a-etl-pipeline-226656a22f6d page

# Extract data from source

In [33]:
# CONSTANT VALUES
OWNER = 'CSSEGISandData'
REPO = 'COVID-19'
PATH = 'csse_covid_19_data/csse_covid_19_daily_reports'
# url = "https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports"
url = f'https://api.github.com/repos/{OWNER}/{REPO}/contents/{PATH}'

In [34]:
response = requests.get(url)

In [35]:
download_urls = []
response = requests.get(URL)
for data in tqdm(response.json()):
    if data['name'].endswith('.csv'):
        download_urls.append(data['download_url'])

  0%|          | 0/1000 [00:00<?, ?it/s]

In [36]:
# creating a progress bar using tqdm
from tqdm.auto import tqdm
import time
# Create a list or iterable to iterate over
my_list = range(10)
# Wrap the iterable with tqdm for progress bar display
for item in tqdm(my_list, desc="Processing"):
    # Simulate some processing time
    time.sleep(0.5)

Processing:   0%|          | 0/10 [00:00<?, ?it/s]

# Transform

In [38]:
labels = ['Province_State', 'Country_Region', 'Last_Update', 'Confirmed', 'Deaths', 'Recovered']
# List of labels to be renamed
relabel = {
    # 'Last Update': 'Last_Update',
    'Country/Region': 'Country_Region',
    'Lat': 'Latitude',
    'Long_': 'Longitude',
    'Province/State': 'Province_State',
}


def factor_dataframe(dat, filename):
    """ Refactor the dataframe to be uploaded into a SQL database
    as a pandas DataFrame
    """
    # rename labels
    for label in dat:
        if label in relabel:
            dat = dat.rename(columns = {label: relabel[label]})
    
    # return a dataframe with these parameters
    labels = ['Province_State', 'Country_Region', 'Last_Update', 'Confirmed', 'Deaths', 'Recovered']
    # filename is datetime
    if 'Last_Update' not in dat:
        dat['Last_Update'] = pd.to_datetime(filename)

    # replace columns not in dataframe with nan
    for label in labels:
        if label not in dat:
            dat[label] = np.nan

    return dat[labels]

# Load to sqlite db

In [39]:
def upload_to_sql(filenames, db_name, debug=False):
    """ Given a list of paths, upload to a database
    """
    conn = sqlite3.connect(f"{db_name}.db")
    
    if debug:
        print("Uploading into database")
    for i, file_path in tqdm(list(enumerate(filenames))):
        
        dat = pd.read_csv(file_path)

        # rename labels
        filename = os.path.basename(file_path).split('.')[0]
        dat = factor_dataframe(dat, filename)

        # write records to sql database
        if i == 0: # if first entry, and table name already exist, replace
            dat.to_sql(db_name, con=conn, index = False, if_exists='replace')
        else: # otherwise append to current table given db_name
            dat.to_sql(db_name, con=conn, index = False, if_exists='append')


# upload into sql database
upload_to_sql(download_urls, 'example', debug=True)

Uploading into database


  0%|          | 0/999 [00:00<?, ?it/s]

# Another ETL 

In [41]:
#https://github.com/habibdraft/tmdb/blob/main/tmdb.py

In [40]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import requests
import json
import config

response_list = []
API_KEY = config.api_key

for movie_id in range(550,556):
    r = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, API_KEY))
    response_list.append(r.json())

df = pd.DataFrame.from_dict(response_list)
genres_list = df['genres'].tolist()
flat_list = [item for sublist in genres_list for item in sublist]

result = []
for l in genres_list:
    r = []
    for d in l:
        r.append(d['name'])
    result.append(r)
df = df.assign(genres_all=result)

df_genres = pd.DataFrame.from_records(flat_list).drop_duplicates()

df_columns = ['budget', 'id', 'imdb_id', 'original_title', 'release_date', 'revenue', 'runtime']
df_genre_columns = df_genres['name'].to_list()
df_columns.extend(df_genre_columns)
s = df['genres_all'].explode()
df = df.join(pd.crosstab(s.index, s))

df['release_date'] = pd.to_datetime(df['release_date'])
df['day'] = df['release_date'].dt.day
df['month'] = df['release_date'].dt.month
df['year'] = df['release_date'].dt.year
df['day_of_week'] = df['release_date'].dt.day_name()
df_time_columns = ['id', 'release_date', 'day', 'month', 'year', 'day_of_week']

df[df_columns].to_csv('tmdb_movies.csv', index=False)
df_genres.to_csv('tmdb_genres.csv', index=False)
df[df_time_columns].to_csv('tmdb_datetimes.csv', index=False)

ModuleNotFoundError: No module named 'config'

In [3]:
url="https://www.google.com/maps/@-17.7954992,31.0270483,14z?entry=ttu"
response = requests.get(url)
# Parse HTML content
soup = BeautifulSoup(response.content, "html.parser")

# Prettify and print the content
print(soup.prettify())

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/Place" lang="en-ZW">
 <head>
  <link as="script" href="/maps/_/js/k=maps.m.en.eBUy3kpm9k0.es5.O/m=sc2,per,mo,lp,ti,ds,stx,dwi,enr,bom,b/am=BgCjAQk/rt=j/d=1/rs=ACT90oH92-K9f-30co2sjyiKn8iMg2fO_Q?wli=m.NI-b_Tk_kI0.loadSv.O%3A%3Bm.Z4ieDLz3MC8.mapcore.O%3A%3B" nonce="4CJZxLL10SJITT0OGYP6Dg" rel="preload" type="application/javascript"/>
  <link href="/maps/preview/opensearch.xml?hl=en" rel="search" title="Google Maps" type="application/opensearchdescription+xml"/>
  <title>
   Google Maps
  </title>
  <meta content="Find local businesses, view maps and get driving directions in Google Maps." name="Description"/>
  <meta content="Anm+hhtuh7NJguqSnXHEAIqqMaV+GXCks8WYXHJKF7l6AeYMj+wO+fi9OdDqFnJTg9t0492DykVxx4jpvFbxnA8AAABseyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IlByaXZhY3lTYW5kYm94QWRzQVBJcyIsImV4cGlyeSI6MTY5NTE2Nzk5OSwiaXNTdWJkb21haW4iOnRydWV9" http-equiv="origin-trial"/>
  <meta content="initial-scale=1.0, max

In [ ]:
# Getting coodinates

import requests

def get_coordinates(place):
    # Replace YOUR_API_KEY with your actual Google Maps API key
    api_key = 'YOUR_API_KEY'
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={place}&key={api_key}'

    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        # Extract the coordinates from the response
        location = data['results'][0]['geometry']['location']
        latitude = location['lat']
        longitude = location['lng']
        return latitude, longitude
    else:
        print('Unable to geocode the place.')
        return None

# Example usage
place = 'New York City'
coordinates = get_coordinates(place)

if coordinates:
    latitude, longitude = coordinates
    print(f'Coordinates of {place}: Latitude={latitude}, Longitude={longitude}')